In [1]:
from fmri.acquisitions import SparklingAcquisition
from fmri.reconstructors.base import SequentialFMRIReconstructor
from mri.operators import WaveletN 
from modopt.opt.linear import Identity
from modopt.opt.proximity import SparseThreshold


/home/pc266769/.pyenv/versions/3.6.13/envs/fmri/lib/python3.6/site-packages/mri/operators/fourier/non_cartesian.py:27: UserWarning: pynfft python package has not been found. If needed use the master release.
  warnings.warn("pynfft python package has not been found. If needed use "


In [2]:
data_file = "/volatile/pierre-antoine/optimed/meas_MID36_2D_Sparkling_1mmiso0_3s_FID3362.dat"
bin_file = "/volatile/pierre-antoine/optimed/dim2_i_RadialIO_P0.75_N192x192_FOV0.192x0.192_Nc6_Ns2688_c30_d2__D7M9Y2021T160.bin"

In [3]:
acq = SparklingAcquisition(data_file,bin_file,frame_slicer=slice(0,2))

pymapVBVD version 0.4.3
Software version: VB


Scan 1/1, read all mdhs:   0%|          | 0.00/15.4G [00:00<?, ?B/s]

.dat file red


/volatile/pierre-antoine/gits/nsp/pysap-fmri/fmri/acquisitions/non_cartesian.py:95: UserWarning: Failed to load the smaps, using self calibration instead
  warnings.warn("Failed to load the smaps, using self calibration instead")


read data:   0%|          | 0/12 [00:00<?, ?it/s]

kspace_data imported
[192 192] (32, 161280) (161280, 2) (2,)


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  32 out of  32 | elapsed:    2.9s finished


In [4]:
acq.kspace_data.shape, acq.kspace_loc.shape, acq.smaps.shape

((2, 32, 80640), (80640, 2), (32, 192, 192))

In [ ]:
fourier_op = acq.get_fourier_operator(implementation='gpuNUFFT')
fourier_op.shape, fourier_op.samples, fourier_op.n_coils
mu=1e-8
regularizer_op = SparseThreshold(Identity(), mu, thresh_type="soft")

linear_op = WaveletN(wavelet_name='sym8',
                    nb_scale=3,
                    dim=2,
                    padding='periodization')


In [ ]:
rec = SequentialFMRIReconstructor(fourier_op, linear_op,regularizer_op,Smaps=acq.smaps,verbose=50,optimizer='pogm')

In [ ]:
res = rec.reconstruct(acq.kspace_data)

In [ ]:
import matplotlib.pyplot as plt

for img in res:
    plt.figure()
    plt.imshow(abs(img))